In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from transformers import DistilBertTokenizerFast, TFDistilBertModel
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate, Input

ModuleNotFoundError: ignored

In [ ]:
data = pd.read_excel("sales_filled_2.xlsx")
data.head(3)

,col_0,entity_employees,sector_naf_level_2,city,emissions_total,emissions_scope1,emissions_scope2,emissions_scope3_reported,emissions_scope3,emissions_post_1,...,emissions_post_23,Description,Niveau_1,latitude,longitude,nom_region,id,population,GDP,entity_sales
0,2,600,6920Z,la creche,1341.410000,699.84,99.5,542.07,542.070000,301.0,...,0.0,Accounting activities,M,46.366,-0.309,Nouvelle-Aquitaine,46.4 : -0.3,9074.0,1.827790e+14,116.894092
1,82,829,8720A,ussel,34542.857140,1927.00,491.0,0.00,32124.857140,1168.0,...,0.0,Social housing for mentally disabled and menta...,Q,45.548,2.309,Nouvelle-Aquitaine,45.5 : 2.3,9555.0,1.924680e+14,159.755285
2,138,505,1101Z,creteil,9745.833333,2054.00,285.0,0.00,7406.833333,765.0,...,0.0,Production of distilled alcoholic beverages,C,48.784,2.455,Île-de-France,48.8 : 2.5,468886.0,9.444870e+15,100.997697


In [ ]:
target_variable = "emissions_total"
selected_columns = ['entity_employees', 'entity_sales', 'Niveau_1', 'population', 'GDP', 'latitude', 'longitude']
X = data[selected_columns]
y = data[target_variable]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=26)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ["entity_employees", "entity_sales"]),
        ('cat', OneHotEncoder(), ["Niveau_1"])
    ])

X_train_processed = preprocessor.fit_transform(X_train).toarray()
X_test_processed = preprocessor.transform(X_test).toarray()


In [ ]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train_processed.shape[1],)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(2, activation='relu'))
model.add(Dense(1))

NameError: ignored

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train_processed, y_train, epochs=100, batch_size=32, validation_split=0.1)

Epoch 1/100
73/73 [==============================] - 2s 9ms/step - loss: 52348634267648.0000 - val_loss: 1544182956032.0000
Epoch 2/100
73/73 [==============================] - 0s 6ms/step - loss: 52341394898944.0000 - val_loss: 1527506796544.0000
Epoch 3/100
73/73 [==============================] - 0s 6ms/step - loss: 52177569579008.0000 - val_loss: 1456840638464.0000
Epoch 4/100
73/73 [==============================] - 0s 7ms/step - loss: 51987198509056.0000 - val_loss: 1507300474880.0000
Epoch 5/100
73/73 [==============================] - 0s 6ms/step - loss: 51879690108928.0000 - val_loss: 1623934107648.0000
Epoch 6/100
73/73 [==============================] - 0s 6ms/step - loss: 51829509455872.0000 - val_loss: 1576982282240.0000
Epoch 7/100
73/73 [==============================] - 0s 6ms/step - loss: 51786530422784.0000 - val_loss: 1559609475072.0000
Epoch 8/100
73/73 [==============================] - 0s 7ms/step - loss: 51771720335360.0000 - val_loss: 1574129631232.0000
Epoch 9/

In [ ]:
mse = model.evaluate(X_test_processed, y_test)
print(f"Mean Squared Error: {mse}")

21/21 [==============================] - 0s 2ms/step - loss: 3738960396288.0000
Mean Squared Error: 3738960396288.0


In [ ]:
y_pred = model.predict(X_test_processed)
y_pred = np.squeeze(y_pred)

21/21 [==============================] - 0s 2ms/step


In [ ]:
r2 = r2_score(y_test, y_pred)
print(f"R-squared: {r2}")

R-squared: 0.2272724265818563
